http://www2.informatik.uni-freiburg.de/~cziegler/BX/

### Summary

### Importing Libraries

In [9]:
import pandas as pd

### Data Cleaning and Splitting

#### Users Dataset

In [10]:
# Read csv data
users = pd.read_csv("assets/input/book_recommendation/users.csv", sep=";", quotechar='"', encoding="latin-1")

In [11]:
# Fill NaN values in the "Location" column with the string "undefined"
users['Location'].fillna("undefined", inplace=True)

In [12]:
# Split the "Location" column using a comma (,) as the separator
location_split = users["Location"].str.split(",", expand=True)

# Assign the split values to the "City," "State/Province," and "Country" columns
users["City"] = location_split[0].str.strip()  # Strip leading/trailing spaces
users["State/Province"] = location_split[1].str.strip()
users["Country"] = location_split[2].str.strip()

# Drop the original "Location" column as it's no longer needed
users.drop("Location", axis=1, inplace=True)

In [13]:
# Step 1: Replace "undefined" with "n/a" in the DataFrame
users.replace("n/a", "undefined", inplace=True)
users

,User-ID,Age,City,State/Province,Country
0,1,NaN,nyc,new york,usa
1,2,18.0,stockton,california,usa
2,3,NaN,moscow,yukon territory,russia
3,4,17.0,porto,v.n.gaia,portugal
4,5,NaN,farnborough,hants,united kingdom
...,...,...,...,...,...
278853,278854,NaN,portland,oregon,usa
278854,278855,50.0,tacoma,washington,united kingdom
278855,278856,NaN,brampton,ontario,canada
278856,278857,NaN,knoxville,tennessee,usa


In [14]:
# Fill NaN values in the "Age" column with the median age
median_age = users['Age'].median()
users['Age'].fillna(median_age, inplace=True)

#### Books Dataset

In [15]:
# Read csv data
books = pd.read_csv("assets/input/book_recommendation/books.csv", encoding="ISO-8859-1", sep=";", error_bad_lines=False)

/Users/iremkurt/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping lin

In [16]:
# Remove rows with NaN values
books.dropna(inplace=True)

# Drop the "Image-URL-S", "Image-URL-M", and "Image-URL-L" columns
books.drop(["Image-URL-S", "Image-URL-M", "Image-URL-L"], axis=1, inplace=True)

#### Book Ratings

In [17]:
# Read csv data
book_ratings = pd.read_csv("assets/input/book_recommendation/book_ratings.csv", encoding="ISO-8859-1", sep=";", error_bad_lines=False)

In [18]:
# Merge users and book_ratings dataframes on "User-ID"
merged_df = pd.merge(users, book_ratings, on='User-ID', how='inner')

# Merge the resulting dataframe with the books dataframe on "ISBN"
recommendation = pd.merge(merged_df, books, on='ISBN', how='inner')

In [20]:
# Add a new column to hold the "Yes" or "No" values
recommendation['Is-Liked?'] = recommendation['Book-Rating'].apply(lambda x: 'Yes' if x >= 7 else 'No')

In [21]:
recommendation

,User-ID,Age,City,State/Province,Country,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Is-Liked?
0,2,18.0,stockton,california,usa,0195153448,0,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,No
1,8,32.0,timmins,ontario,canada,0002005018,5,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,No
2,11400,49.0,ottawa,ontario,canada,0002005018,0,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,No
3,11676,32.0,undefined,undefined,undefined,0002005018,8,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,Yes
4,41385,32.0,sudbury,ontario,canada,0002005018,0,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,No
...,...,...,...,...,...,...,...,...,...,...,...,...
1031124,278851,33.0,dallas,texas,usa,0743203763,0,As Hogan Said . . . : The 389 Best Things Anyo...,Randy Voorhees,2000,Simon &amp; Schuster,No
1031125,278851,33.0,dallas,texas,usa,0767907566,5,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001,Broadway Books,No
1031126,278851,33.0,dallas,texas,usa,0884159221,7,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985,Lone Star Books,Yes
1031127,278851,33.0,dallas,texas,usa,0912333022,7,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997,Kqed Books,Yes
